## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Omboue,GA,-1.5746,9.2618,75.90,92,99,5.79,overcast clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,35.11,64,100,4.74,overcast clouds
2,2,Ribeira Grande,PT,38.5167,-28.7000,61.03,73,56,19.08,broken clouds
3,3,Hermanus,ZA,-34.4187,19.2345,55.90,81,25,4.92,light rain
4,4,Severo-Kurilsk,RU,50.6789,156.1250,37.53,98,100,21.41,light rain


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Omboue,GA,-1.5746,9.2618,75.90,92,99,5.79,overcast clouds
11,11,Dibombari,CM,4.1787,9.6561,75.24,95,100,1.05,overcast clouds
12,12,Samarai,PG,-10.6167,150.6667,79.03,74,72,18.54,broken clouds
14,14,Bengkulu,ID,-3.8004,102.2655,75.24,92,100,3.78,overcast clouds
17,17,Mayumba,GA,-3.4320,10.6554,76.23,92,85,3.76,overcast clouds
28,28,Hit,IQ,33.6416,42.8251,87.40,14,0,4.88,clear sky
32,32,Atuona,PF,-9.8000,-139.0333,78.84,72,92,9.48,light rain
51,51,Acapulco,MX,16.8634,-99.8901,87.75,61,58,7.40,broken clouds
52,52,Kapaa,US,22.0752,-159.3190,82.11,71,44,1.99,scattered clouds
61,61,Guaratinga,BR,-16.5667,-39.5667,75.87,92,76,2.04,broken clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                203
City                   203
Country                203
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Omboue,GA,75.90,overcast clouds,-1.5746,9.2618,
11,Dibombari,CM,75.24,overcast clouds,4.1787,9.6561,
12,Samarai,PG,79.03,broken clouds,-10.6167,150.6667,
14,Bengkulu,ID,75.24,overcast clouds,-3.8004,102.2655,
17,Mayumba,GA,76.23,overcast clouds,-3.4320,10.6554,
28,Hit,IQ,87.40,clear sky,33.6416,42.8251,
32,Atuona,PF,78.84,light rain,-9.8000,-139.0333,
51,Acapulco,MX,87.75,broken clouds,16.8634,-99.8901,
52,Kapaa,US,82.11,scattered clouds,22.0752,-159.3190,
61,Guaratinga,BR,75.87,broken clouds,-16.5667,-39.5667,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")

hotel_df.head(10)    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Omboue,GA,75.90,overcast clouds,-1.5746,9.2618,Hotel Olako
11,Dibombari,CM,75.24,overcast clouds,4.1787,9.6561,
12,Samarai,PG,79.03,broken clouds,-10.6167,150.6667,Nuli Sapi
14,Bengkulu,ID,75.24,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
17,Mayumba,GA,76.23,overcast clouds,-3.4320,10.6554,LIKWALE LODGE
28,Hit,IQ,87.40,clear sky,33.6416,42.8251,صالون زيرو
32,Atuona,PF,78.84,light rain,-9.8000,-139.0333,Villa Enata
51,Acapulco,MX,87.75,broken clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
52,Kapaa,US,82.11,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
61,Guaratinga,BR,75.87,broken clouds,-16.5667,-39.5667,Hotel Caraíva


In [11]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"].str.len() > 2)]

clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Omboue,GA,75.90,overcast clouds,-1.5746,9.2618,Hotel Olako
12,Samarai,PG,79.03,broken clouds,-10.6167,150.6667,Nuli Sapi
14,Bengkulu,ID,75.24,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
17,Mayumba,GA,76.23,overcast clouds,-3.4320,10.6554,LIKWALE LODGE
28,Hit,IQ,87.40,clear sky,33.6416,42.8251,صالون زيرو


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 

max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))